<a href="https://colab.research.google.com/github/paskef/llm-weather/blob/main/weather_llm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install and import

In [443]:
!pip install -q -U google-generativeai

In [444]:
import google.generativeai as genai
import requests
import json
import re

### Set the API Key

In [445]:

from google.colab import userdata

GOOGLE_API_KEY=userdata.get('api_key')
genai.configure(api_key=GOOGLE_API_KEY)
OPENWEATHER_API_KEY = userdata.get('openweather_api_key')

### Model settings

In [446]:
generation_config_result = {
    'candidate_count': 1,
    'temperature': 0.2,
}

In [447]:
generation_config = {
    'candidate_count': 1,
    'temperature': 0,
}

In [448]:
safety_config = [
    {
        'category': 'HARM_CATEGORY_HARASSMENT',
        'threshold': 'BLOCK_MEDIUM_AND_ABOVE'
    },
    {
        'category': 'HARM_CATEGORY_HATE_SPEECH',
        'threshold': 'BLOCK_MEDIUM_AND_ABOVE'
    },
    {
        'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT',
        'threshold': 'BLOCK_MEDIUM_AND_ABOVE'
    },
    {
        'category': 'HARM_CATEGORY_DANGEROUS_CONTENT',
        'threshold': 'BLOCK_MEDIUM_AND_ABOVE'
    },
]

In [449]:
model = genai.GenerativeModel(model_name='gemini-1.0-pro',
                              generation_config=generation_config,
                              safety_settings=safety_config)


### Get latitude and longitude

In [450]:
history = []
while True:
  try:
    location = input('Enter the location: ')
    history.append(f'User: Enter the location: {location}\n')
    response = model.generate_content(f'What are the latitude and longitude from {location}?')
    print(response.text)
    generated_text = response.text
    numbers = re.findall(r'[-+]?\d*\.?\d+', response.text)
    latitude = numbers[0]
    longitude = numbers[1]
    break
  except IndexError:
    print('Invalid input. Please enter a valid location.')


Enter the location: São Paulo Brazil
-23.5475° S, -46.6361° W


In [451]:
history.append(f'Model: The location is {location}. What are the latitude and longitude?\n')
history.append(f'Latitude: {latitude} Longitude: {longitude} \n')

In [452]:

url = f'http://api.openweathermap.org/data/2.5/weather?lat={latitude}&lon={longitude}&appid={OPENWEATHER_API_KEY}&units=metric'

response = requests.get(url)
data = response.json()

In [453]:
temp = data['main']['temp']
weather_conditions = data['weather']

### Generate results

In [454]:
model = genai.GenerativeModel(model_name='gemini-1.0-pro',
                              generation_config=generation_config_result,
                              safety_settings=safety_config)

In [455]:
response = model.generate_content(f'Temperature in {location} is {temp}°C with {weather_conditions}. List the location, temperature, weather conditions, possible clothing combinations (in topics) and why did you choose them.')

In [456]:
print(response.text)

**Location:** São Paulo, Brazil

**Temperature:** 21.34°C

**Weather Conditions:** Overcast clouds

**Possible Clothing Combinations:**

* **Tops:**
    * Long-sleeved shirt
    * Light sweater
    * Thin jacket
* **Bottoms:**
    * Jeans
    * Khakis
    * Dress pants
* **Footwear:**
    * Closed-toe shoes
    * Boots
    * Sneakers

**Reasoning:**

The temperature is mild, but the overcast clouds indicate that it may rain. Therefore, it is advisable to wear layers of clothing that can be easily adjusted as needed. A long-sleeved shirt or light sweater will provide warmth, while a thin jacket can be added if the temperature drops. Jeans or khakis are comfortable and appropriate for the weather conditions. Closed-toe shoes or boots will protect the feet from any rain or moisture.


#### *History settings*

In [457]:
history.append(f'Model: Temperature in {location} is {temp}°C with {weather_conditions}. List the location, temperature, weather conditions, possible clothing combinations (in topics) and why did you choose them\n')
history.append(response.text)
with open('chat_history.md', 'w') as f:
    for item in history:
        f.write("%s\n" % item)